# Pre-processing pipeline

In [ ]:
import mne
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import pywt
import scipy as sp

### configuration setup

In [ ]:
RAW_PATH = '/Users/jannis/Git/tuh-eeg-seizure-detection/data/raw'
OUTPUT_PATH = '/Users/jannis/Git/tuh-eeg-seizure-detection/data/processed_fast'

SAMPLING_FREQ = 250
WINDOW_LENGTH = 10
CONFIGURATIONS = []
CHANNELS = []

### load raw data

#### extract events from annotations

In [ ]:
def extract_events_from_annotations(annotation_file):
    with open(annotation_file, "r") as f:
        annotations = f.readlines()
        events = annotations[6:] 
        
        data = []
        for event in events:
            parts = event.split(",")
            
            start_time = float(parts[1])
            stop_time = float(parts[2])
            label = parts[3]
            
            data.append({
                "label": label,
                "onset": start_time,
                "duration": stop_time - start_time
            })
            
            
    return data

#### load the TUH EEG dataset

In [ ]:
def load_tuh_eeg():
    cols = ["set", "patient_id", "session_id", "configuration", "recording_id", "recording_path", "label", "onset", "duration"]
    data = []
    
    # get all edf files in RAW/edf
    edf_path = os.path.join(RAW_PATH, "edf")
    for root, dirs, files in os.walk(edf_path):
        for file in files:
            if file.endswith(".edf"):
                rel_path = os.path.relpath(root, edf_path)
                parts = rel_path.split("/")
                
                if len(parts) != 4:
                    continue
                    
                set_name, patient_id, session_id, configuration = parts
                
                recording_path = os.path.join(root, file)
                recording_id = file.replace(".edf", "").split("_")[-1]
                annotation_path = recording_path.replace(".edf", ".csv_bi")
                
                if not os.path.exists(recording_path) or not os.path.exists(annotation_path):
                    continue
                
                events = extract_events_from_annotations(annotation_path)
                for event in events:
                    data.append({
                        "set": set_name,
                        "patient_id": patient_id,
                        "session_id": session_id,
                        "configuration": configuration,
                        "recording_id": recording_id,
                        "recording_path": recording_path,
                        "label": event["label"],
                        "onset": event["onset"],
                        "duration": event["duration"]
                    })
                    
    return pd.DataFrame(data, columns=cols)

data = load_tuh_eeg()
data.tail()

### feature extraction

In [ ]:
def calc_coeffs_features(coeffs):
    mean = np.mean(coeffs)
    median = np.median(coeffs)
    std = np.std(coeffs)
    skew = sp.stats.skew(coeffs)
    kurtosis = sp.stats.kurtosis(coeffs)
    rms = np.sqrt(np.mean(coeffs**2))
    
    return {
        "mean": mean,
        "median": median,
        "std": std,
        "skew": skew,
        "kurtosis": kurtosis,
        "rms": rms
    } 
    
def extract_wavelet_features(channel_data):
    cA5, cD5, cD4, cD3, cD2, cD1 = pywt.wavedec(channel_data, 'db4', level=5)
    
    d3_features = calc_coeffs_features(cD3)
    d4_features = calc_coeffs_features(cD4)
    d5_features = calc_coeffs_features(cD5)
    a5_features = calc_coeffs_features(cA5)
    
    # rename keys to include the level
    d3_features = {f"d3_{k}": v for k, v in d3_features.items()}
    d4_features = {f"d4_{k}": v for k, v in d4_features.items()}
    d5_features = {f"d5_{k}": v for k, v in d5_features.items()}
    a5_features = {f"a5_{k}": v for k, v in a5_features.items()}
    
    features = {}
    features.update(a5_features)
    features.update(d3_features)
    features.update(d4_features)
    features.update(d5_features) 
     
    return features


### pre-process the data

In [ ]:
def preprocess(patient_id):
    corrupted = []
    
    # output directory
    seizure_output_dir = os.path.join(OUTPUT_PATH, patient_id, "seizure")
    non_seizure_output_dir = os.path.join(OUTPUT_PATH, patient_id, "non_seizure")
    os.makedirs(seizure_output_dir, exist_ok=True)
    os.makedirs(non_seizure_output_dir, exist_ok=True)
    
    recordings = data[data["patient_id"] == patient_id]["recording_path"].unique()
    
    for recording in recordings:
        raw = mne.io.read_raw_edf(recording, preload=True).pick_channels(CHANNELS)
        
        # sometimes meas date breaks code
        raw.set_meas_date(None)
        
        # pre-processing to remove noise
        raw.resample(SAMPLING_FREQ)
    
        events = data[data["recording_path"] == recording]
        
        onset = events["onset"].values
        duration = events["duration"].values
        label = events["label"].values
        
        annotations = mne.Annotations(onset=onset, duration=duration, description=label)
        raw.set_annotations(annotations)
        
        for i, event in events.iterrows():
            patient_id = event["patient_id"]
            onset = event["onset"]
            duration = event["duration"]
            label = event["label"]
            
            num_windows = int(duration / WINDOW_LENGTH)
            
            if num_windows == 0:
                continue
                
            if onset + duration > raw.times[-1]:
                if onset + duration - 1/SAMPLING_FREQ == raw.times[-1]:
                    raw_event = raw.copy().crop(onset, raw.times[-1], include_tmax=True)
                else:
                    print("Corrupted annotation", patient_id, event["session_id"], event["recording_id"])
                    corrupted.append((patient_id, event["session_id"], event["recording_id"]))
                    continue
            else:
                raw_event = raw.copy().crop(onset, onset + duration, include_tmax=False)
                
            epochs = mne.make_fixed_length_epochs(raw_event, duration=WINDOW_LENGTH)
            
            for epoch in epochs:
                for channel in epoch:
                    features = extract_wavelet_features(channel)
                    print(features)
                
            output_dir = seizure_output_dir if label == "seiz" else non_seizure_output_dir
            file_name = f"{patient_id}_{event['session_id']}_{event["recording_id"]}_{i}_epo.fif"
            epochs.save(os.path.join(output_dir, file_name), overwrite=True)
            raw_event.close()
            
        raw.close()
        
    return corrupted

In [ ]:
patients = data["patient_id"].unique()
patients = patients[200:210]
patient = patients[0]
preprocess(patient)

In [ ]:
len(data["patient_id"].unique())